In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
import sys, os
import missingno as msno

In [ ]:
%reload_ext watermark
%watermark -a "Leonardo da Silva Neves"

In [ ]:
# Carregando DataFrames das edições do censo da educação superior.
censo_24 = 'censo_2024.csv'
file_ies = 'censo_ies/MICRODADOS_ED_SUP_IES_2024.CSV'

In [ ]:
pd.set_option('display.max_columns', 200)

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
pd.set_option('display.max_colwidth', 100)

In [ ]:
# Abrindo o arquivo como tipo binário 'rb' - 'read binary'
with open(censo_24, 'rb') as f1:
    raw_data = f1.read(1000000)

In [ ]:
# Descobrindo a codificação do dataset com o método .detect() da biblioteca chardet.
result_23 = chardet.detect(raw_data)
cod_23 = result_23['encoding']
conf_23 = result_23['confidence']
print(f'codificação detectada: {cod_23} (confiança: {conf_23:.2f}%)')

In [ ]:
df_24 = pd.read_csv(censo_24, sep=';', encoding=cod_23, low_memory=False, decimal=',')
df_24.shape

In [ ]:
cols = ['CO_IES', 'NO_REGIAO_IES', 'CO_REGIAO_IES', 'NO_UF_IES', 'SG_UF_IES', 'CO_UF_IES', 'NO_MUNICIPIO_IES', 'CO_MUNICIPIO_IES', 'IN_CAPITAL_IES']

In [ ]:
ies_24 = pd.read_csv(file_ies, sep=';', encoding=cod_23, low_memory=False, decimal=';', usecols=cols)
ies_24.shape

In [ ]:
(df_24.isna().mean()*100).round(2).head()

In [ ]:
# Selecionando apenas colunas com valores numéricos
colunas_quantidade = [col for col in df_24.columns if col.startswith('QT_')]
len(colunas_quantidade)

In [ ]:
# Preechendo os valores nulos (NaN) para evitar erros nas operações matemáricas, usando 0 nas colunas de contagem
for col in colunas_quantidade:
    df_24[col] = df_24[col].fillna(0)

# Solução alternativa usando .loc
# df_11.loc[:,colunas_quantidade] = df_11[colunas_quantidade].fillna(0)

Preenchendo os valores ausentes da coluna **'TP_GRAU_ACADEMICO'**

In [ ]:
# 1. vou criar uma série (mascara booleana) (True onde é nan)
mask_na = df_24['TP_GRAU_ACADEMICO'].isna()

In [ ]:
# 2. Aplicar a mascara boleanda para selecionar apenas as linhas com valor True
g_acadna = df_24[mask_na]

Para começar a preencher os valores ausentes da coluna 'TP_GRAU_ACADEMICO', algumas decisões de análise precisarão ser tomadas.

Para reduzir o número de valores ausentes na coluna 'TP_NIVEL_ACADEMICO'

In [ ]:
# 3. Verificando os valores únicos da variável 'TP_NIVEL_ACADEMICO'
g_acadna['TP_NIVEL_ACADEMICO'].unique()

In [ ]:
# 4. Criando uma mascara booleanda apenas com os cursos Sequencial de formação especifica.
mask_cs = g_acadna['TP_NIVEL_ACADEMICO'] == 2 # Sequencial de formação específica

In [ ]:
# 5. Aplicando a mascara booleana para para selecionar apenas as linha com valor True da variável 'TP_NIVEL_ACADEMICO'
c_fesp = g_acadna[mask_cs]

In [ ]:
c_fesp.shape

In [ ]:
# 6. Verificando as incidências de valores da variável 'NO_CINE_AREA_GERAL'
c_fesp['NO_CINE_AREA_GERAL'].value_counts()

In [ ]:
# 7. Criando uma cópia do DataFrame para realizar as imputações
df_2024 = df_24.copy()

In [ ]:
# 7.1 Checando as dimensões do novo DF
df_2024.shape

In [ ]:
# 7.2 Verificando os valores ausentes na variável alvo.
df_2024['TP_GRAU_ACADEMICO'].isna().sum()

In [ ]:
# 7.3 Criando uma validação condicional utilizando as duas mascaras criadas anteriormente e atribuindo um novo valor para a variável alvo.
condition = (mask_na & mask_cs)
df_2024.loc[condition, 'TP_GRAU_ACADEMICO'] = 5 #  Sequencial de formação específica

In [ ]:
# 7.4 Conferindo a redução no número valores ausentes na variável alvo
df_2024['TP_GRAU_ACADEMICO'].isna().sum()

In [ ]:
# 8. Criando uma nova mascara de seleção dos valores ausentes remanecentes na variável 'TP_GRAU_ACADEMICO'
mask1_na = df_2024['TP_GRAU_ACADEMICO'].isna()

In [ ]:
# 8.1 Criação de um novo df filtrado pela mascara criada anteriormente
g_acadna1 = df_2024[mask1_na]

In [ ]:
# 8.2 Verificação da dimensão do df olhando apenas para a quantidade de linhas
g_acadna1.shape[0]

In [ ]:
# 8.3 Verificando algumas amostras de linhas do DataFrame para verificar a insidência de algum padrão
g_acadna1.sample(5)

In [ ]:
# 8.4 Analisando algumas estatísticas das variáveis não númericas do DataFrame
g_acadna1.describe(include = 'object')

In [ ]:
#  8.4.1 Verificando as regiões do Brasil que não preencheram a variável ´TP_GRAU_ACADEMICO´ para o cursos diferentes de ...
# 'Sequencial de formação especifica"
g_acadna1['NO_REGIAO'].value_counts()

In [ ]:
# 8.4.2 Verificando insidência de valores ausentes por tipo de Rede (Pública e Privada)
g_acadna1['TP_REDE'].value_counts()

In [ ]:
# 8.4.3 Visualizando a ocorrência dos cursos para determinar quais são bacharelados por definição e quais não são.
g_acadna1['NO_CURSO'].value_counts()

In [ ]:
g_acadna1['NO_CURSO'].unique()

In [ ]:
# 8.4.4 Criando uma mascara classificando os cursos de licenciatura e bacharelado
bacharelado = [
    'Abi - Design',
    'Abi - Engenharia',
    'Abi - Engenharia De Minas',
    'Abi - Engenharia Elétrica',
    'Abi Ciclo Básico Minas/Petróleo',
    'Abi - Ciclo Básico Materiais/Metalúrgica/Nuclear',
    'Engenharia Metalúrgica E Engenharia De Materiais',
    'Abi - Ciência Da Computação',
    'Computação',
    'Abi - Comunicação Social',
    'Abi - Ciências Biomédicas',
    'Abi - Economia Empresarial E Controladoria',
    'Abi - Psicologia',
    'Antropologia E Arqueologia',
    'Abi - Enfermagem'
]
len(bacharelado)

In [ ]:
# 8.4.4.2 Criando a condição lógica para definir a atribuição de valores para as outras variáveis
lic_bac = [
    'Abi - Educação Física',
    'Abi - Artes Cênicas',
    'Abi - Ciências Sociais',
    'Abi - Filosofia',
    'Abi - Geografia',
    'Abi - História',
    'Abi - Matemática',
    'Abi - Química',
    'Abi - Educação Intercultural',
    'Abi - Letras',
    'Abi - Licenciaturas Interdisciplinares',
    'Abi - Ciências Biológicas',
    'Abi - Dança',
    'Abi - Física',
    'Abi - Teatro',
    'Abi - Licenciatura Intercultural',
    'Abi - Artes Visuais',
    'Abi - Ciências Exatas',
    'Abi - Ciclo Básico Composição/Regência',
    'Abi - Educação Física E Esporte',
    'Abi - Matemática Aplicada',
    'Abi - Matemática Aplicada E Computacional',
    'Abi - Física/Engenharia Física', # Física permite L/B
    'Abi - Física, Matemática/Matemática Aplicada E Computacional',
    'Abi - Ciências Da Natureza: Ciências E Biologia, Ciências E Física Ou Ciências E Química',
    'Abi - Ciências Da Natureza E Matemática',
    'Abi - Letras - Língua Portuguesa',
    'Abi - Letras Francês',
    'Abi - Letras Inglês',
    'Abi - Música',
    'Abi - Letras Português',
    'Abi - Letras Alemão',
    'Abi - Letras Espanhol',
    'Abi - Letras Italiano',
    'Abi - Letras Língua Portuguesa',
    'Abi - Letras Vernáculas',
    'Abi - Letras Vernáculas E Língua Estrangeira Moderna',
    'Abi - Língua Estrangeira Moderna Ou Clássica',
    'Abi - Música Popular',
    'Abi - Letras - Português - Arabe',
    'Abi - Letras - Português - Espanhol',
    'Abi - Letras - Português - Frances',
    'Abi - Letras - Português - Hebraico',
    'Abi - Letras - Português - Ingles',
    'Abi - Letras - Português - Italiano',
    'Abi - Letras - Português - Latim',
    'Abi - Letras - Português - Russo',
    'Abi - Letras Português - Alemão',
    'Abi - Literaturas De Língua Portuguesa',
    'Matemática - Licenciatura Ou Bacharelado',
    'Ciências Biológicas - Licenciatura Ou Bacharelado',
    'Ciências Sociais - Licenciatura Ou Bacharelado',
    'Geografia - Licenciatura Ou Bacharelado',
    'História - Licenciatura Ou Bacharelado',
    'Química - Licenciatura Ou Bacharelado'
]

In [ ]:
# 8.4.4.1 Criando a condição lógica para definir a atribuição de valores
mask2 = df_2024['NO_CURSO'].isin(bacharelado)
mask3 = df_2024['TP_GRAU_ACADEMICO'].isna()
mask4 = df_2024['NO_CURSO'].isin(lic_bac)
condiction1 = (mask2 & mask3)
condiction2 = (mask4 & mask3)

In [ ]:
# Conferindo número de atribuições da condiction1
condiction1.sum()

In [ ]:
df_2024['TP_GRAU_ACADEMICO'].isna().sum()

In [ ]:
# 8.4.4.3 Atribuição para os registros que atendem a condição de não licenciaturas
df_2024.loc[condiction1, 'TP_GRAU_ACADEMICO'] = 1 # Bacharelado

In [ ]:
df_2024['TP_GRAU_ACADEMICO'].isna().sum()

In [ ]:
# 8.4.4.4 Atribuição para os registros que atendem a condição de licenciaturas
df_2024.loc[condiction2, 'TP_GRAU_ACADEMICO'] = 4 # Bacharelado ou licenciaturas

In [ ]:
# Verificando o número de atribuições da condiction2
condiction2.sum()

In [ ]:
# 9. Verificação da incidência de valores nulos na variável 'TP_GRAU_ACADEMICO'
df_2024['TP_GRAU_ACADEMICO'].isna().sum()

In [ ]:
df_tgau = df_2024['TP_GRAU_ACADEMICO'].isna()

In [ ]:
df = df_2024[df_tgau]

In [ ]:
df['NO_CURSO'].unique()

**Tratamento das demais variáveis com valores ausentes**

['NO_REGIAO', 'CO_REGIAO', 'NO_UF', 'SG_UF', 'CO_UF', 'NO_MUNICIPIO', 'CO_MUNICIPIO', 'IN_CAPITAL']

In [ ]:
(df_2024.isna().mean()*100).round(2)

Aplicação da função .map() para otimização **'NO_REGIAO'**

In [ ]:
# 1. Verificando as dimensões do df lookup (Capastro_ies - fonte de mapeamento)
ies_24.shape

In [ ]:
# 1.2 Verificando se existe dados duplicados na variável chave do df lookup
ies_24['CO_IES'].duplicated().sum()

In [ ]:
# 2. Criação da série de mapeamento (dicionário de lookup) index= CO_IES (chave) Values = 'NO_REGIAO'
map_cat1 = ies_24.set_index('CO_IES')['NO_REGIAO_IES']
# O método .drop_duplicates() é essencial para garantir que a chave (CO_IES) seja única, evitando que o .map() falhe ou retorne valores incorretos.
#map_cat1 = map_cat1.drop_duplicates() # no caso atual, não é necessário pois a os registro do df ies_10 são 1 para 1.

In [ ]:
map_cat1.head()

In [ ]:
# Gerando uma nova serie com os valores do mapeamento
cat_map = df_2024['CO_IES'].map(map_cat1)

In [ ]:
df_2024['NO_REGIAO'] = df_2024['NO_REGIAO'].fillna(cat_map)

In [ ]:
df_2024['NO_REGIAO'].isna().sum()

Aplicando a função na variável **'CO_REGIAO'**

In [ ]:
# 1. Criação da série de mapeamentos (lookup)
map_cat2 = ies_24.set_index('CO_IES')['CO_REGIAO_IES']

In [ ]:
# 2. Gerando série com os valores do mapeamento
cat_map2 = df_2024['CO_IES'].map(map_cat2) 

In [ ]:
# 3. Preechimento de valores com .fillna()
df_2024['CO_REGIAO'] = df_2024['CO_REGIAO'].fillna(cat_map2)

In [ ]:
# 4. Validando a aplicação
df_2024['CO_REGIAO'].isna().sum()

Aplicando a função na variável **'NO_UF'**

In [ ]:
# 1. Criação da série de mapeamento (lookup)
map_cat3 = ies_24.set_index('CO_IES')['NO_UF_IES']

In [ ]:
# 2. Gerando série com os valores do mapeamento
cat_map3 = df_2024['CO_IES'].map(map_cat3)

In [ ]:
# 3. Preenchimento de valores com .fillna()
df_2024['NO_UF'] = df_2024['NO_UF'].fillna(cat_map3)

In [ ]:
# 4. Validação da aplicação
df_2024['NO_UF'].isna().sum()

Aplicando a função na variável **'SG_UF'**

In [ ]:
# 1. Criação da série de mapeamento (lookup)
map_cat4 = ies_24.set_index('CO_IES')['SG_UF_IES']

In [ ]:
# 2. Gerando série com os valores do mapeamento
cat_map4 = df_2024['CO_IES'].map(map_cat4)

In [ ]:
# 3. Preenchimento de valores com .fillna()
df_2024['SG_UF'] = df_2024['SG_UF'].fillna(cat_map4)

In [ ]:
# 4. Validação da aplicação
df_2024['SG_UF'].isna().sum()

Aplicando a função na variável **'CO_UF'**

In [ ]:
df_2024.CO_UF.isna().sum()

In [ ]:
# 1. Criação de série de mapeamento (lookup)
map_cat5 = ies_24.set_index('CO_IES')['CO_UF_IES']

In [ ]:
# 2. Gerando série com os valores do mapemento
cat_map5 = df_2024['CO_IES'].map(map_cat5)

In [ ]:
# 3. Preechimento de valores com .fillna()
df_2024['CO_UF'] = df_2024['CO_UF'].fillna(cat_map5)

In [ ]:
# 4. Validação da aplicação
df_2024.CO_UF.isna().sum()

Aplicando a função na variável **'NO_MUNICIPIO'**

In [ ]:
df_2024.NO_MUNICIPIO.isna().sum()

In [ ]:
# 1. Criação de série de mapeamento (lookup)
map_cat6 = ies_24.set_index('CO_IES')['NO_MUNICIPIO_IES']

In [ ]:
# 2. Gerando série com os valores do mapeamento
cat_map6 = df_2024['CO_IES'].map(map_cat6)

In [ ]:
# 3. Preenchimento de valores com fillna()
df_2024['NO_MUNICIPIO'] = df_2024['NO_MUNICIPIO'].fillna(cat_map6)

In [ ]:
# 4. Validação
df_2024['NO_MUNICIPIO'].isna().sum()

Aplicando a função na variável **'CO_MUNICIPIO'**

In [ ]:
df_2024.CO_MUNICIPIO.isna().sum()

In [ ]:
# 1. Criação de série de mapeamento (lookup)
map_cat7 = ies_24.set_index('CO_IES')['CO_MUNICIPIO_IES']

In [ ]:
# 2. Gerando série com os valores do mapeamento
cat_map7 = df_2024['CO_IES'].map(map_cat7)

In [ ]:
# 3. Preenchimento de valores com fillna()
df_2024['CO_MUNICIPIO'] = df_2024['CO_MUNICIPIO'].fillna(cat_map7)

In [ ]:
# 4. Validação
df_2024['CO_MUNICIPIO'].isna().sum()

Aplicando a função na variável **'CO_MUNICIPIO'**

In [ ]:
df_2024.IN_CAPITAL.isna().sum()

In [ ]:
# 1. Criação de série de mapeamento (lookup)
map_cat8 = ies_24.set_index('CO_IES')['IN_CAPITAL_IES']

In [ ]:
# 2. Gerando série com os valores do mapeamento
cat_map8 = df_2024['CO_IES'].map(map_cat8)

In [ ]:
# 3. Preenchimento de valores com fillna()
df_2024['IN_CAPITAL'] = df_2024['IN_CAPITAL'].fillna(cat_map8)

In [ ]:
# 4. Validação
df_2024['IN_CAPITAL'].isna().sum()

#### Aplicação de imputação condicional para para preencher os valores ausentes da variável 'TP_REDE' 

In [ ]:
# 1. Definindo os valores da variável que serão usadas para validação
#ies_publicas = [1, 2, 3] # TP_CATEGORIA_ADMINISTRATIVA (PÚBLICA)
# 2. Definindo a condição de nulidade da variável TP_REDE
#cond_na = df_2013['TP_REDE'].isna()
# 3. Definindo a condição da coluna secundária
#cond_sec = df_2013['TP_CATEGORIA_ADMINISTRATIVA'].isin(ies_publicas)
# 4. Combinando as duas condições (NA e [1, 2, 3])
#conditions = (cond_na & cond_sec)
# 5. Aplicando a alteração (imputação condicional)
#valor = 1
#df_2013.loc[conditions, 'TP_REDE'] = valor

In [ ]:
# print(f" Aplicado em {conditions.sum()} registros")

In [ ]:
# 1. Definindo  os valores da variável que será usada para validação
#ies_privadas = [4, 5, 6, 7, 8, 9]
# 2. Condição de nulidade
#cond_na2 = df_2013['TP_REDE'].isna()
# 3. Coluna segundária
#cond_sec2 = df_2013['TP_CATEGORIA_ADMINISTRATIVA'].isin(ies_privadas)
# 4. Combinando as duas condições
#condictions2 = (cond_na2 & cond_sec2)
# 5. Aplicar a Alteração
#valor2 = 2
#df_2013.loc[condictions2, 'TP_REDE'] = valor2

In [ ]:
#print(f"Aplicado em {condictions2.sum()} registros")

In [ ]:
# Restam valores ausentes?
#df_2013['TP_REDE'].isna().sum()

In [ ]:
# Existem outras variáveis com valores ausentes?
(df_2024.isna().mean()*100).round(2)

**Engenharias em 2024**

In [ ]:
# 1. Criando uma máscara booleana usando .str.contains()
mask_case_insensitive = df_2024['NO_CURSO'].str.contains('engenharia', case=False, na=False)
# 2. Aplicando a máscara para filtragem
df_eng_24 = df_2024[mask_case_insensitive]

In [ ]:
df_eng_24.shape

In [ ]:
df_eng_ead = df_eng_24[df_eng_24['TP_MODALIDADE_ENSINO'] == 2]

In [ ]:
df_eng_ead.shape

In [ ]:
# Salvando em .csv
# df_2024.to_csv('censo_2024.csv', sep=';', encoding=cod_23, index=False)

### Fim